# Computer Vision을 위한 Transfer Learning

일반적으로 VGG, ResNet, gooGleNet 등 이미 이러한 사전에 학습이 완료된 모델(Pre-Training Model)을 가지고 우리가 원하는 데이터를 학습시키는 방법이 Transfer Learning이다.

이러한 전이학습 시나리오의 주요한 2가지는 다음과 같다:

* 합성곱 신경망의 미세조정(finetuning): 무작위 초기화 대신, 신경망을 ImageNet 1000 데이터셋 등으로 미리 학습한 신경망으로 초기화한다. 학습의 나머지 과정들은 평상시와 같다.

* 고정된 특징 추출기(fixed feature extractor)로써의 합성곱 신경망: 여기서는 마지막에 Fully connected layer를 제외한 모든 신경망의 가중치를 고정한다. 이 마지막의 Fully connected layer는 새로운 임의의 가중치를 갖는 계층으로 대체되어 이 계층만 학습하게 된다.


**Transfer learning 장점**
* 적은 데이터로도 학습이 가능
* 학습 속도도 빠르다. 
* 예측을 더 높이는 방법이다.

# 과제

여기서 풀고자 하는 문제는 개미 와 벌 을 분류하는 모델을 학습하는 것이다. 개미와 벌 각각의 학습용 이미지는 대략 120장 정도 있고, 75개의 검증용 이미지가 있다. 일반적으로 맨 기초부터(from scratch) 학습을 한다면 이는 일반화하기에는 아주 작은 데이터셋이다. 하지만 우리는 전이학습을 이용하면 일반화를 제법 잘 할 수 있을 것이다.

[data download](https://download.pytorch.org/tutorial/hymenoptera_data.zip)

**Pretrained Model은 resnet18을 사용한다.**
```
model_conv = torchvision.models.resnet18(pretrained=True)
```

다음의 순서로 실습을 진행한다.

1. Data를 Load 한다. 제공되는 hymenoptera_data.zip을 다운로드하여 현재 디렉토리에 압축해제 한다.
2. Finetuning the convnet
    - 미리 학습한 모델을 불러온 후
    - 마지막 FC layer를 데이터에 맞게 변경한다.
    - 해당 모형의 가중치를 초기화 값으로 하여 해당 데이터를 학습한다.
3. ConvNet as fixed feature extractor
    - 미리 학습한 모델을 불러온 후
    - requires_grad = False 로 설정하여 매개변수를 고정하여 backward() 중에 gradient가 계산되지 않도록 고정한다.
    - 마지막 FC layer를 데이터에 맞게 변경하여 학습을 진행한다.
4. 2번에서 학습된 모형과 3번에서 학습된 모형을 비교해 본다.


**그 밖의 하이퍼파라미터, 손실함수, 옵티마이저 등은 자유롭게 선택하여 학습을 진행한다.**

참고) https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import os

In [2]:
os.environ['TORCH_HOME'] = '../models' # pretrained model의 저장위치

data_dir = 'hymenoptera_data/' # 데이터 저장 위치
train_dir = os.path.join(data_dir, 'train/') # train data 
test_dir = os.path.join(data_dir, 'val/') # test data

In [3]:
model_conv = torchvision.models.resnet18(pretrained=True)